In [64]:
def isEmpty(string):
    if string == "" or string == None:
        return True
    return False

#### Create an 'enum' for header (column) numbering of the dataframe

In [65]:
keys = ["HS Hdg", 
        "HS Code", 
        "Blank",
        "Description", 
        "Unit",
        "ICL/SLSI",
        "Preferential Duty_AP",
        "Preferential Duty_AD",
        "Preferential Duty_BN",
        "Preferential Duty_GT",
        "Preferential Duty_IN",
        "Preferential Duty_PK",
        "Preferential Duty_SA",
        "Preferential Duty_SF",
        "Preferential Duty_SD",
        "Preferential Duty_SG",
        "Gen Duty",
        "VAT",
        "PAL_Gen",
        "PAL_SG",
        "Cess_GEN",
        "Cess_SG",
        "Excise SPD",
        "SSCL",
        "SCL"]

values = list(range(0,25))
headerNumber = dict(zip(keys, values))

#### Get notes and tables from the pdf

In [66]:
import pdfplumber

allText = []
rows = []
tableReached = False

pdf = pdfplumber.open('Ch. 28.pdf')

for page in pdf.pages:
    if tableReached: # if table has already been reached, continue to extract table, no need to find more text due to the document structure having all required text before the table
        rowsPerPage = page.extract_table()
        for row in rowsPerPage:
            rows.append(row)
        continue

    tableInPage = page.find_table()
    if tableInPage == None: # if no table found, extract text from whole page
        textPerPage = page.extract_text()
        allText.append(textPerPage)
    else: # else extract text from the part above the table, and start extracting the table
        tableBBox = tableInPage.bbox
        tableTop = tableBBox[1] # top value is 2nd value of bounding box tuple
        croppedPage = page.within_bbox((0,0,page.width,tableTop)) # bounding box tuple order: left, top, right, bottom
        textPerPage = croppedPage.extract_text()
        allText.append(textPerPage)
        tableReached = True
        rowsPerPage = page.extract_table()
        for row in rowsPerPage:
            rows.append(row)

pdf.close()

import pandas as pd
df = pd.DataFrame(rows)

    

#### Initialize Dictionaries for json

In [67]:

dictionaryForThisPDF = dict.fromkeys(["Document Name", "Pre-Table Notes", "Items"])



In [68]:
dictionaryForThisPDF["Document Name"] = 'Ch. 28.pdf'
preTableNotes = ""
for text in allText:
    preTableNotes += text
dictionaryForThisPDF["Pre-Table Notes"] = preTableNotes

In [69]:
import json

# json_string = json.dumps(dictionaryForThisPDF)

# Get table row

In [104]:
# df.head(30)
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,HS Hdg,HS Code,,Description,Unit,ICL/\nSLSI,Preferential Duty,None,None,None,...,None,Gen\nDuty,VAT,PAL,None,Cess,None,Excise\n(S.P.D),SSCL,S\nC\nL
1,None,None,None,None,None,None,AP,AD,BN,GT,...,SG,None,None,Gen,SG,GEN,SG,None,None,None
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,I.- CHEMICAL ELEMENTS,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,None,None,None,None,None,None,AP,AD,BN,GT,...,SG,None,None,Gen,SG,GEN,SG,None,None,None
404,,,,,,,,,,,...,,,,,,,,,,
405,,2853.10,-,Cyanogen chloride (Chlorcyan),kg,L,,,,,...,Free,Free,18%,Ex,,,,,2.5%,
406,,2853.90,-,Other,kg,,,,,,...,Free,Free,18%,Ex,,,,,2.5%,


In [110]:
ongoing_prefix = ""
ongoing_hshdgname = ""
ongoing_hshdg = ""
keysForAnItem = ["Prefix", "HS Hdg Name","HS Hdg","HS Code","Blank", "Description", "Unit","ICL/SLSI","Preferential Duty_AP","Preferential Duty_AD","Preferential Duty_BN","Preferential Duty_GT","Preferential Duty_IN","Preferential Duty_PK","Preferential Duty_SA","Preferential Duty_SF","Preferential Duty_SD","Preferential Duty_SG","Gen Duty","VAT","PAL_Gen","PAL_SG","Cess_GEN","Cess_SG","Excise SPD","SSCL","SCL"]
items = []

numOfRows = df.shape[0] # rows
# only a row with a non-null unit will be considered a valid item
for n in range(3,numOfRows):
    current_hshdg = df.loc[n].values[headerNumber['HS Hdg']]
    current_hscode = df.loc[n].values[headerNumber['HS Code']]
    current_description = df.loc[n].values[headerNumber['Description']]
    current_unit = df.loc[n].values[headerNumber['Unit']]

    if isEmpty(current_description) or (current_hshdg == "HS Hdg") or (current_hshdg == None): # row is considered empty
        continue
    if isEmpty(current_hshdg) and isEmpty(current_unit): # description considered a prefix
        ongoing_prefix = current_description
        continue
    if (not isEmpty(current_hshdg)) and isEmpty(current_hscode): # row has a HS Hdg no. and no HS code no.
        ongoing_hshdg = current_hshdg
        ongoing_hshdgname = current_description
        ongoing_prefix = "" # reset on-going prefix since we have moved to a new hs hdg
        if isEmpty(current_unit): #not an item
            continue
        else:
            current_hscode = current_hshdg # becomes a valid item
    if not isEmpty(current_unit): # this is a valid item
        # create a json item
        values = [ongoing_prefix] + [ongoing_hshdgname] + list(df.loc[n].values)
        item = dict(zip(keysForAnItem, values))
        item.pop('Blank',item['Blank'])
        item['HS Hdg'] = ongoing_hshdg
        item['HS Code'] = current_hscode # in case this is an item that has a hs hdg, but no declared hs code
        items.append(item)
        

    

In [115]:
items

[{'Prefix': '',
  'HS Hdg Name': 'Flourine, chlorine, bromine and iodine',
  'HS Hdg': '28.01',
  'HS Code': '2801.10',
  'Description': 'Chlorine',
  'Unit': 'kg',
  'ICL/SLSI': '',
  'Preferential Duty_AP': '',
  'Preferential Duty_AD': '',
  'Preferential Duty_BN': '',
  'Preferential Duty_GT': '',
  'Preferential Duty_IN': 'Free',
  'Preferential Duty_PK': 'Free',
  'Preferential Duty_SA': '',
  'Preferential Duty_SF': 'Free',
  'Preferential Duty_SD': 'Free',
  'Preferential Duty_SG': 'Free',
  'Gen Duty': 'Free',
  'VAT': '18%',
  'PAL_Gen': '10%',
  'PAL_SG': '6%',
  'Cess_GEN': '',
  'Cess_SG': '',
  'Excise SPD': '',
  'SSCL': '2.5%',
  'SCL': ''},
 {'Prefix': '',
  'HS Hdg Name': 'Flourine, chlorine, bromine and iodine',
  'HS Hdg': '28.01',
  'HS Code': '2801.20',
  'Description': 'Iodine',
  'Unit': 'kg',
  'ICL/SLSI': '',
  'Preferential Duty_AP': '',
  'Preferential Duty_AD': '',
  'Preferential Duty_BN': '',
  'Preferential Duty_GT': '',
  'Preferential Duty_IN': 'Free',

In [109]:
df.to_csv('ch28.csv')

In [116]:
type(items)

list

In [118]:
dictionaryForThisPDF["Items"] = items

In [119]:
print(dictionaryForThisPDF["Items"])

[{'Prefix': '', 'HS Hdg Name': 'Flourine, chlorine, bromine and iodine', 'HS Hdg': '28.01', 'HS Code': '2801.10', 'Description': 'Chlorine', 'Unit': 'kg', 'ICL/SLSI': '', 'Preferential Duty_AP': '', 'Preferential Duty_AD': '', 'Preferential Duty_BN': '', 'Preferential Duty_GT': '', 'Preferential Duty_IN': 'Free', 'Preferential Duty_PK': 'Free', 'Preferential Duty_SA': '', 'Preferential Duty_SF': 'Free', 'Preferential Duty_SD': 'Free', 'Preferential Duty_SG': 'Free', 'Gen Duty': 'Free', 'VAT': '18%', 'PAL_Gen': '10%', 'PAL_SG': '6%', 'Cess_GEN': '', 'Cess_SG': '', 'Excise SPD': '', 'SSCL': '2.5%', 'SCL': ''}, {'Prefix': '', 'HS Hdg Name': 'Flourine, chlorine, bromine and iodine', 'HS Hdg': '28.01', 'HS Code': '2801.20', 'Description': 'Iodine', 'Unit': 'kg', 'ICL/SLSI': '', 'Preferential Duty_AP': '', 'Preferential Duty_AD': '', 'Preferential Duty_BN': '', 'Preferential Duty_GT': '', 'Preferential Duty_IN': 'Free', 'Preferential Duty_PK': 'Free', 'Preferential Duty_SA': '', 'Preferentia

In [120]:
dictionaryForThisPDF

{'Document Name': 'Ch. 28.pdf',
 'Pre-Table Notes': 'Chapter 28\nInorganic chemicals; organic or inorganic compounds\nof precious metals, of rare-earth metals,\nof radioactive elements or of isotopes\nNotes.\n1.- Except where the context otherwise requires, the headings of this Chapter apply only to :\n(a) Separate chemical elements and separate chemically defined compounds, whether or not containing impurities;\n(b) The products mentioned in (a) above dissolved in water;\n(c) The products mentioned in (a) above dissolved in other solvents provided that the solution constitutes a normal and necessary method of putting up these products adopted solely for reasons of safety or for transport and that\nthe solvent does not render the product particularly suitable for specific use rather than for general use;\n(d) The products mentioned in (a), (b) or (c) above with an added stabiliser (including an anti-caking agent) necessary for their preservation or transport;\n(e) The products mentione

#### Save to json file

In [122]:
json_string = json.dumps(dictionaryForThisPDF)
with open('ch28.json','w') as file:
    file.write(json_string)
